## Online Statistics

In [1]:
import numpy as np
import logging 

In [2]:
class OnlineStatistics(object):
    def __init__(self, order):
        """
        Parameters
        ----------
        order: int, the number of attributes
        """
        self._order = order
        self._shape = (order, order)
        self._identity = np.identity(order)
        self._count = 0
        self._mean = np.zeros(order)
        self._cov = np.zeros(self._shape)
    
    def update(self, X):
        if X.shape[1] != self._order:
            logging.warning("Incorrect dimension")
            return 
             
        if X.shape[0] == 1:
            self._update_stats(X.flatten())
        else:
            for x in X:
                self._update_stats(x)
    
    def _update_stats(self, observation):
        """
        Incremental update on mean and covariance matrix
        Original source https://carstenschelp.github.io/2019/05/12/Online_Covariance_Algorithm_002.html
        
        Parameters
        ----------
        observation: array_like, The observation to add.
        """
        if self._order != len(observation):
            raise ValueError(f'Observation to add must be of size {self._order}')
         
        # update the number of objects seen so far
        self._count += 1
        # compute the difference between the given object and observed mean
        delta_at_nMin1 = np.array(observation - self._mean)
        
        # update mean
        self._mean += delta_at_nMin1 / self._count
        
        weighted_delta_at_n = np.array(observation - self._mean) / self._count
        D_at_n = np.broadcast_to(weighted_delta_at_n, self._shape)
        
        ## diagonal matrix
        D = (delta_at_nMin1 * self._identity).dot(D_at_n)
        self._cov = self._cov * (self._count - 1) / self._count + D
        
    @property
    def cov(self):
        return self._cov
    
    @property
    def mean(self):
        return self._mean